<a href="https://colab.research.google.com/github/muhozag/sparklearning/blob/logreg/logregconsult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting Church with Logistic Regression**

In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName('logisticregconsult').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager',
                                       'Years', 'Num_Sites'],
                             outputCol='features')

In [8]:
output = assembler.transform(data)

In [9]:
final_data = output.select('features', 'churn')

In [10]:
train_data, test_data = final_data.randomSplit([0.75, 0.25])

In [11]:
train_data.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                685|
|   mean| 0.1678832116788321|
| stddev|0.37403566193483045|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [12]:
test_data.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                215|
|   mean|0.16279069767441862|
| stddev|0.37003615371102505|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
lr_churn = LogisticRegression(labelCol='churn')

In [15]:
fitted_church_model = lr_churn.fit(train_data)

In [16]:
fitted_church_model.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                685|                685|
|   mean| 0.1678832116788321|0.13576642335766423|
| stddev|0.37403566193483045| 0.3427906685401636|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_and_labels = fitted_church_model.evaluate(test_data)

In [18]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.57044902450284...|[0.63886677875243...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.59428674023282...|[0.99629472905258...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.50856265048773...|[0.81884809402997...|       0.0|
|[28.0,11128.95,1....|    0|[4.19570673664853...|[0.98516334623885...|       0.0|
|[28.0,11245.38,0....|    0|[3.83331836849124...|[0.97882057931321...|       0.0|
|[29.0,11274.46,1....|    0|[4.50854744841820...|[0.98910554874760...|       0.0|
|[29.0,13255.05,1....|    0|[4.10374852146153...|[0.98375750577858...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.29043119397538...|[0.96409908160842...|       0.0|
|[30.0,11575.37,1....|    1|[3.99365160657536...|[0.98190131643696...|       0.0|
|[31.0,5387.75,0

In [19]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

In [20]:
AUC = churn_eval.evaluate(pred_and_labels.predictions)

In [21]:
AUC

0.7317460317460317

In [25]:
final_lr_model = lr_churn.fit(final_data)

In [26]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [27]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [28]:
test_new_customers = assembler.transform(new_customers)

In [29]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [30]:
final_results = final_lr_model.transform(test_new_customers)

In [31]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

